In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [3]:
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [4]:
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [5]:
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [6]:
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [7]:
transactions.sort_values(['store_nbr','date'])

,date,store_nbr,transactions
1,2013-01-02,1,2111
47,2013-01-03,1,1833
93,2013-01-04,1,1863
139,2013-01-05,1,1509
185,2013-01-06,1,520
...,...,...,...
83271,2017-08-11,54,768
83325,2017-08-12,54,903
83379,2017-08-13,54,1054
83433,2017-08-14,54,818


In [8]:
# Date-time conversions

train['date'] = pd.to_datetime(train.date)
test['date'] = pd.to_datetime(test.date)
transactions['date'] = pd.to_datetime(transactions.date)
oil['date'] = pd.to_datetime(oil.date)

In [9]:
# Initial plots

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

px.line(transactions.sort_values(["store_nbr", "date"]), x='date', y='transactions', color='store_nbr',title = "Transactions" )

In [10]:
transactions['year'] = transactions.date.dt.year
transactions['month'] = transactions.date.dt.month

px.box(transactions, x = 'year', y = 'transactions', color = 'month', title = 'Store transactions - Monthly patterns')

In [11]:
# Day of week patterns

a = transactions.copy()
a['dayofweek'] = a.date.dt.dayofweek+1

a = a.groupby(['year', 'dayofweek']).transactions.mean().reset_index()
px.line(a, x = 'dayofweek', y = 'transactions', color = 'year', title = 'Avg transactios - By day of week ')

In [12]:
px.line(oil, x = 'date', y = 'dcoilwtico', title = 'Oil price trend' )

In [13]:
# Resampling & Interpolating for missing values

oil = oil.set_index("date").dcoilwtico.resample("D").sum().reset_index()

oil['dcoilwtico'] = np.where(oil['dcoilwtico'] == 0, np.nan, oil['dcoilwtico'])
oil['dcoilwtico_interpolated'] = oil.dcoilwtico.interpolate()

px.line(oil, x = 'date', y = 'dcoilwtico_interpolated', title = 'Oil price trend (Interpolated data)' )

In [14]:
# Correlation between sales & txns

temp = pd.merge(train.groupby(["date", "store_nbr"]).sales.sum().reset_index(), transactions, how = "left")
print("Spearman Correlation between Total Sales and Transactions: {:,.4f}".format(temp.corr("spearman").sales.loc["transactions"]))


Spearman Correlation between Total Sales and Transactions: 0.8175


In [15]:
# Correlation between oil prices and sales / txns

temp = pd.merge(temp, oil, how = 'left')
print("Correlations between oil prices and sales / txns")
print(temp.drop(['dcoilwtico','store_nbr'], axis = 1).corr("spearman").dcoilwtico_interpolated.loc[['sales', 'transactions']])

Correlations between oil prices and sales / txns
sales          -0.303237
transactions    0.040105
Name: dcoilwtico_interpolated, dtype: float64
